### 3.0 LLMs and Chat Models

In [1]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
llm = OpenAI(model_name="gpt-3.5-turbo-1106")
chat = ChatOpenAI()

/Users/sigi/hanwha_project/langchain_study/kang/env/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/sigi/hanwha_project/langchain_study/kang/env/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [3]:
a = llm.predict("How many planets are there?")
b = llm.predict("How many planets are there?")

In [4]:
a, b

('As of now, there are 8 recognized planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Additionally, there are 5 dwarf planets recognized by the International Astronomical Union: Pluto, Eris, Haumea, Makemake, and Ceres. However, there may be other planets or dwarf planets that have not yet been discovered or classified.',
 'There are eight officially recognized planets in the Solar System: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. Additionally, there is ongoing discussion and debate about the status of Pluto as a planet, as it was reclassified as a "dwarf planet" by the International Astronomical Union in 2006.')